In [1]:
import pandas as pd 
import numpy as np 
import os

In [2]:
def data_generator():
    
    directories = os.listdir('/kaggle/input/icecube-neutrinos-in-deep-ice/train/')
    directories = directories[0:1]#temporal
    directories = [('/kaggle/input/icecube-neutrinos-in-deep-ice/train/')+x for i, x in enumerate(directories)]
    
    data = pd.DataFrame()
    
    
    for file in directories:
        temp = pd.read_parquet(file)
        
        
        data = pd.concat([data, temp], axis=0)
        
    
    coords = pd.read_csv('/kaggle/input/icecube-neutrinos-in-deep-ice/sensor_geometry.csv')
    train_meta = pd.read_parquet('/kaggle/input/icecube-neutrinos-in-deep-ice/train_meta.parquet')
    
    data['event'] = data.index
    
    data = pd.merge(data, train_meta[['event_id', 'azimuth', 'zenith']], left_on=data['event'], right_on=train_meta['event_id'].astype(float))
    
    data = pd.merge(data, coords, on='sensor_id', how='outer')


    data = pd.concat([data.groupby('event')['time'].apply(list),
                   data.groupby('event')['charge'].apply(list),
                   data.groupby('event')['x'].apply(list),
                   data.groupby('event')['y'].apply(list),
                   data.groupby('event')['z'].apply(list),
                   data.groupby('event')['azimuth'].apply(min),
                   data.groupby('event')['zenith'].apply(min)],axis=1)
    
    return data[['time','charge','x','y','z','azimuth','zenith']]


In [3]:
data = data_generator()

In [4]:
data.head(3)

,time,charge,x,y,z,azimuth,zenith
event,,,,,,,
777938857.0,"[6029.0, 6078.0, 6508.0, 7886.0, 8003.0, 8479....","[1.4249999523162842, 1.475000023841858, 0.4250...","[-200.55, 544.07, -166.4, 106.94, 330.03, -189...","[-74.03, 55.89, -287.79, 27.09, 127.2, 257.42,...","[-225.49, -24.86, 157.24, -174.78, -384.21, 36...",0.095967,2.747172
777938862.0,"[6866.0, 7551.0, 7899.0, 7907.0, 16245.0, 7941...","[1.524999976158142, 0.9750000238418579, 0.3249...","[330.03, -413.46, -132.8, 576.37, 576.37, -234...","[127.2, -327.27, -501.45, 170.92, 170.92, 140....","[-94.86, 74.08, 278.8, -237.85, -237.85, -163....",2.533461,0.638181
777938866.0,"[11231.0, 11243.0, 12541.0, 10299.0, 6130.0, 6...","[2.075000047683716, 1.024999976158142, 0.625, ...","[-200.55, -200.55, -413.46, 282.18, 35.54, -12...","[-74.03, -74.03, -327.27, -325.74, -364.83, -1...","[-225.49, -225.49, 329.39, -488.76, -132.05, -...",2.510193,1.059650


In [ ]:
input = keras.Input(shape=(7,))

x = layers.Flatten()(input)
x = layers.Dense(200, activation='softmax')(x)
x = layers.Dense(200, activation='softmax')(x)

out_az = keras.layers.Dense(10, activation='softmax', name='az-out')(x)
out_ze = keras.layers.Dense(10, activation='softmax', name='ze-out')(x)


encoder = keras.Model( inputs = input, outputs = [out_az, out_ez], name="encoder")


encoder.compile(
    loss = {
        'az-out': tf.keras.losses.CategoricalCrossentropy(),
        'ze-out': tf.keras.losses.CategoricalCrossentropy(),
    },

    metrics = {
        'az-out': 'accuracy',
        'ze-out': 'accuracy',
    },

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    

encoder.fit(xtrain, [y_az, y_ze], epochs=10, verbose=2)
    

In [9]:
sample #submission

,event_id,azimuth,zenith
0,2092,1,1
1,7344,1,1
2,9482,1,1
